# Developing a classifier for houses of multiple or single occupancy

In [ ]:
# Load required python libraries
import sqlite3
import numpy as np
import pandas as pd
import datetime
import statsmodels.api as sm
from statsmodels.tsa.api import VAR
import matplotlib.pyplot as plt
from functions import count_events_per_interval, fit_VAR_model

# There is a pandas FutureWarning about using catagorical variables in groupby. Pandas will address this in future updates.
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Attach sqlite data base
connection = sqlite3.connect("../data/data.db")
cursor = connection.cursor()

## Import data from Sqlite database to pandas data frame
Sqlite database consists of two tables; homes and motion. The total database is only about 60 Mb in size so it can be loaded straight into memory (via a pandas dataframe) without overflow concerns.

### homes

homes holds whether the home is a single or multiple occupancy house.

- id is the unique house id, renamed home_id (32 hexadecimal)
- multiple_occupancy is whether house is occupied by more than one person (boolean)

### motion

motion holds each motion detection event as a tuple with four entries

- id is the unique event id, renamed event_id (32 hexadecimal)
- home_id is the unique home id (hexadecimal)
- datetime is the time of the motion detection event (YYYY-MM-DD HH:MM:SS+ss)
- location is the room of house that the motion was detected in (category: living room, conservatory, dining room, study, WC1, hallway, bathroom1, kitchen, lounge, bedroom1)

In [ ]:
homes = pd.read_sql_query("SELECT * FROM homes", connection)

# Specify multiple_occupancy as boolean
homes["multiple_occupancy"]=homes["multiple_occupancy"].astype('bool')

# Change id to home_id to be consistent and specific
homes = homes.rename(columns={"id":"home_id"})

In [ ]:
motion = pd.read_sql_query("SELECT * FROM motion", connection)

# Specify datetime should be a datetime variable
motion["datetime"] = pd.to_datetime(motion["datetime"])

# Specify location should be a category (This gives a FutureWarning in groupby commands that pandas will address soon)
motion["location"] = motion["location"].astype('category')

# Change id to event_id to be consistent and specific
motion = motion.rename(columns={"id":"event_id"})

In [ ]:
motion_homes_join = pd.read_sql_query("""SELECT motion.id as `event_id`, motion.home_id, motion.datetime, motion.location, homes.multiple_occupancy
FROM motion
INNER JOIN homes ON motion.home_id=homes.id;""", connection)

## Initial data exploration

- **is there any missing data** No detected null/na values in imported data.
- **do all home_id's have entries in both tables?**     Of the 106 unique homes in the home data set, there are only 50 unique homes with motion data.
- **are all the events unique?**     There are 580317 events, every event has a unique id.
- **do all homes have the same rooms?**     No, between 2 and 8.
- **are location names unique?** No, mosts homes have lounges but 3 have lounges and living rooms. Most bathrooms, 2 only have WC, some have both.
- **do the events cover the same time period?** Mosts homes have events covering the time period 1st Jan 2024 until 31st Jan 2024. However, one home only has seven days worth of data.
- **are there similar number of events per house?** No, 1000< x <30,000

In [ ]:
homes.info()

In [ ]:
motion.info()

In [ ]:
motion_homes_join.info()

In [ ]:
print("Number of unique home_ids in homes table: ", len(homes["home_id"].unique()))
print("Number of unique home_ids in motion table: ", len(motion["home_id"].unique()))
print("Number of unique home_ids in motion-homes combined: ", len(motion_homes_join["home_id"].unique()))

In [ ]:
# Check all home ids are 32 digit hexadecimals
print("All home_id's in the homes table are 32 digit hexadecimals:",len(homes["home_id"]) == sum(homes.home_id.str.match("[0-9abcdef]{32}")))
print("All home_id's in the motion table are 32 digit hexadecimals:",len(motion["home_id"]) == sum(motion.home_id.str.match("[0-9abcdef]{32}")))

In [ ]:
# Check all event ids are 32 digit hexadecimals
print("All event_id's in the motion table are 32 digit hexadecimals:",len(motion["home_id"]) == sum(motion.event_id.str.match("[0-9abcdef]{32}")))

In [ ]:
print("Number of unique event_ids in motion table: ", len(motion["event_id"].unique()))

In [ ]:
rooms_per_home = motion[["home_id","location"]].groupby(["home_id","location"]).head(1).groupby(["home_id"]).count()
print("Homes have between ",min(rooms_per_home["location"])," and ",max(rooms_per_home["location"])," rooms.")
events_per_home = motion[["home_id","event_id"]].groupby(["home_id"]).count()
print("Homes have between ",min(events_per_home["event_id"])," and ",max(events_per_home["event_id"])," events.")

In [ ]:
print("Number of houses with each room type:")
motion[["home_id","location"]].groupby(["home_id","location"]).head(1).groupby(["location"]).count().sort_values("home_id")

In [ ]:
print("Number of events per room type:")
motion[["location","event_id"]].groupby(["location"]).count().sort_values("event_id")

In [ ]:
homes_with_living_rooms = motion[motion.location == "living room"]["home_id"].unique()
homes_with_lounges = motion[motion.location == "lounge"]["home_id"].unique()
print(len(list(set(homes_with_living_rooms) & set(homes_with_lounges))), "homes have living rooms and lounges")
print("42 homes only have a lounge")
print("No homes only have a living room")

In [ ]:
homes_with_WC = motion[motion.location == "WC1"]["home_id"].unique()
homes_with_bathrooms = motion[motion.location == "bathroom1"]["home_id"].unique()
print(len(list(set(homes_with_bathrooms) & set(homes_with_WC))), "homes have bathroom and water closets")
print("2 homes only have water closets")
print("29 homes only have bathrooms")

In [ ]:
# Time period of sensor acquisition
home_sensor_last_entry = motion[["home_id","datetime"]].groupby("home_id").max().rename(columns={"datetime":"enddate"})
home_sensor_first_entry = motion[["home_id","datetime"]].groupby("home_id").min().rename(columns={"datetime":"startdate"})
home_sensor_last_first_combined = pd.merge(home_sensor_first_entry, home_sensor_last_entry, on="home_id")
home_sensor_last_first_combined["difference"] = home_sensor_last_first_combined["enddate"] - home_sensor_last_first_combined["startdate"]
print("The shorted period of sensor data is: ", min(home_sensor_last_first_combined["difference"]))

## Data cleaning
To simplify the time series modelling I am going to remove all the event data associated with living room and WC1 locations. All but two homes will still have sensor data in similar rooms. Further exploration is required to determine if these location labels are errors or if these are large homes with multiple receptions rooms/ensuites or if there are multiple sensors in the same rooms. If model performance in these homes is poor we can return to this assumption.

Only a handful of homes have studies/conservatories/dining rooms. For this exploratory analysis I am not going to fit time series models to these data sets. I will add binary labels "has study", "has conservatory", "has dining rooms" as this suggests larger houses with possible multiple occupants. This will give the ML classification some extra data. If model performance in these homes is poor we can return to this assumption.

In order to facilitate the fitting of time series models, I will convert the events based recording to events per 1 day intervals across the entire month of Janurary. I am going to remove the homes with recording periods less than 20 days worth of events detected. The time series models used run on a rolling average. The sensor may have only been installed on the 24th of Jan for one house. The other may have gone on holiday from the 16th, for example. The model will not be able to account for such behaviour.

In [ ]:
# Remove duplicate rooms and rooms found in less than 10 homes
events_selected_locations = motion[motion.location.str.contains('bedroom1|lounge|bathroom1|hallway|kitchen')]

# Remove the two homes with less than 20 days worth of data
events_selected_locations_homes = events_selected_locations[~events_selected_locations.home_id.str.contains('df9f7afaae7821246e296a41e9e2a6b4|15663392d490688cd4b0e5aa3d5b6ef3')]

In [ ]:
homes_with_binary_locations = motion[motion.location.str.contains('conservatory|dining room|study')][["home_id","location"]].groupby(["home_id", "location"]).head(1)

homes_with_binary_locations["has_conservatory"] = homes_with_binary_locations["location"] == "conservatory"

homes_with_binary_locations["has_dining_room"] = homes_with_binary_locations["location"] == "dining room"

homes_with_binary_locations["has_study"] = homes_with_binary_locations["location"] == "study"

homes_with_binary_locations = homes_with_binary_locations.drop(columns = "location")

In [ ]:
# Check if any homes still have any time series data post-filter
events_per_home_post_filter = events_selected_locations_homes[["home_id","event_id"]].groupby(["home_id"]).count()
print("Homes have between ",min(events_per_home_post_filter["event_id"])," and ",max(events_per_home_post_filter["event_id"])," events.")

In [ ]:
# FutureWarning is unactionable and will be solved by pandas team in future
counted_events_per_house = events_selected_locations_homes.groupby(["home_id","location"]).apply(count_events_per_interval, start = "2024-01-01", end = "2024-02-01", interval = "1d", include_groups=False)

# Tidy up dataframe index
counted_events_per_house = counted_events_per_house.reset_index(level=['home_id','location']).reset_index(drop=True)

## Time Series Modeling

A time series model will be fitted to the sensor data of the 5 room types that are common to most homes. The events are discrete integers always greater than 1. Ideally a poisson or neg-bin model should be used for this data. However, a vector autoregressive model with poisson noise would have to be developed (there are limited available python packages for this). 

Instead, I will use a more common vector autoregressive model with gaussian noise for this inital task. Obviously, the continuous noise model with values below zero is not ideal and should be revisited if the model is to be improved. The fact that the data contains a significant number of zeros likely affects the assumption that time series data is stationary. The lag term should be optimised, but as this assessment is not about model performance I shall leave that.

In [ ]:
# Visualise count data
test_count_subset_bed = counted_events_per_house[(counted_events_per_house['home_id'] == 'f5a2b27c9d8bb7f59b7b0684d3555e52') & (counted_events_per_house['location'] == 'bedroom1')]['event_count']
test_count_subset_lounge = counted_events_per_house[(counted_events_per_house['home_id'] == 'f5a2b27c9d8bb7f59b7b0684d3555e52') & (counted_events_per_house['location'] == 'lounge')]['event_count']
fig, ax = plt.subplots(nrows=2, ncols=1)  # Create a figure containing a single axes.
ax[0].plot(range(31), test_count_subset_bed)  # Plot some data on the axes.
ax[0].title.set_text("Bedroom1")
ax[1].plot(range(31), test_count_subset_lounge)  # Plot some data on the axes.
ax[1].title.set_text("Lounge")

In [ ]:
# Check vector autoregression model forcasts resonable results
test_model_fit = counted_events_per_house[(counted_events_per_house['home_id'] == 'f5a2b27c9d8bb7f59b7b0684d3555e52')].pivot_table(index="datetime",values="event_count", columns="location")
#test_model_fit = np.log(test_model_fit)
model = VAR(test_model_fit.iloc[0:29], freq = "D")
results = model.fit(2)
results.forecast(test_model_fit.values[28:], 2)
results.plot_forecast(2)

In [ ]:
# Apply model to all homes
modelled_event_counts = counted_events_per_house.groupby("home_id").apply(fit_VAR_model, lag=2, freq="D", include_groups=False)
# Move index to column names
modelled_event_counts = modelled_event_counts.reset_index(level='home_id')
modelled_event_counts["coefficient"] = modelled_event_counts.index
modelled_event_counts = modelled_event_counts.reset_index(drop=True)
# Pivot all coefficients to column values and flatten column name hierchy 
modelled_event_counts = modelled_event_counts.pivot(columns="coefficient", index="home_id")
modelled_event_counts.columns = [' '.join(col).strip() for col in modelled_event_counts.columns.values]
# Set missing vector autoregression coefficients to 0
modelled_event_counts = modelled_event_counts.fillna(0)
# Join results with binary data to prepare for classification
modelled_event_counts = modelled_event_counts.join(homes_with_binary_locations.set_index('home_id')).fillna(False)

## Classification

## Conclusions